In [ ]:
import pandas as pd

zooniverse_df = pd.read_csv("./ai-for-artefacts-in-sky-images-classifications (5).csv")
zooniverse_df  = zooniverse_df[zooniverse_df['workflow_id']==25237]

In [ ]:
import numpy as np

np.unique(zooniverse_df['user_name']), np.unique(zooniverse_df['workflow_id'])

In [ ]:
zooniverse_df = zooniverse_df[zooniverse_df['workflow_id']==25237]
zooniverse_df

convert csv string column to list

In [ ]:
import json
zooniverse_df['annotations'] = zooniverse_df['annotations'].apply(lambda x:json.loads(x))

In [59]:
for i in range (len(zooniverse_df)):
    subj_data = zooniverse_df['subject_data'][i]#['Filename']
    for k in zooniverse_df['subject_data'][i].keys():
        dict_val = zooniverse_df['subject_data'][i][k]
        # print(dict_val)
        if 'Observation' in dict_val.keys():
            # print(zooniverse_df['subject_data'][i][k]['Observation'])
            if 'S0404420101_L' in zooniverse_df['subject_data'][i][k]['Observation']:
                print(i)

172


In [39]:
zooniverse_df['subject_data'][0]

{'458030': {'retired': None,
  'link': 'https://www.flickr.com/photos/alishav/3278443166',
  'origin': 'Flickr',
  'license': 'Creative Commons - share adapt attribute',
  'image_name': '3278443166_9eecc2ca0b_z.jpg',
  'subject_id': 10,
  'attribution': 'Alisha Vargas'}}

In [ ]:
zooniverse_df['subject_data'] = zooniverse_df['subject_data'].apply(lambda x:json.loads(x))

In [ ]:
zooniverse_df['annotations'][0][0].keys()

In [ ]:
task_labels = []

for annot in zooniverse_df['annotations']:
    if annot[0]['task_label'] not in task_labels:
        task_labels.append(annot[0]['task_label'])

In [ ]:
task_labels

In [ ]:
k=0
for i in range(len(zooniverse_df['annotations'])):
    if zooniverse_df['annotations'][i][0]['task_label'] == 'Does the image have artefacts in it?' and \
     zooniverse_df['annotations'][i][0]['value'] in ['The image has artefacts. ', 'The image has **NO** artefacts.']:
        k+=1
        if len(zooniverse_df['annotations'][i])==1: # usually 'The image has **NO** artefacts.'
            print(zooniverse_df['annotations'][i][0]['value'], zooniverse_df['subject_data'][i])

In [ ]:
from dataset import dataset_utils
import cv2

In [ ]:
no_artefacts_images= []
images_dir = '../XMM_OM_dataset/zscaled_512_stretched/'

In [ ]:
for i in range(len(zooniverse_df['annotations'])):
    if zooniverse_df['annotations'][i][0]['task_label'] == 'Does the image have artefacts in it?' and \
     zooniverse_df['annotations'][i][0]['value'] in ['The image has artefacts. ', 'The image has **NO** artefacts.']:
        subject_data_key = list(zooniverse_df['subject_data'][i])[0] # Python 3.7 or later
        subject_data_key = zooniverse_df['subject_data'][i][subject_data_key]
        # print(subject_data_key)
        if 'metadata' in subject_data_key:
            image_name = subject_data_key['metadata'] 
        elif 'Observation' in subject_data_key:
            image_name = subject_data_key['Observation'] 
            annot = zooniverse_df['annotations'][i]
            if len(annot) <=1: # usually no annotations in the image or 'The image has **NO** artefacts.'
                if image_name not in no_artefacts_images:
                    no_artefacts_images.append(image_name)
            else:
                print(annot)
                image = cv2.imread(images_dir+image_name+'.png')
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                masks, gt_classes, boxes = [], [], []
                for i in range(len(annot[1]['value'])):
                    value_key = annot[1]['value'][i]
                    tool_label = value_key['tool_label']
                    tool = value_key['tool']
                    frame = value_key['frame']
                    details = value_key['details']


In [ ]:
# value
# 0:central-ring
# 1:read-out-streak
# 2:smoke-ring
# 3:star-loop
# 4:other

In [ ]:
values_dict = {
    0:'central-ring',
    1:'read-out-streak',
    2:'smoke-ring',
    3:'star-loop',
    4:'other'}

In [ ]:
import matplotlib.pyplot as plt
import cv2
k=0

images_dict = {}

images_dir = '../XMM_OM_dataset/zscaled_512_stretched/'
no_artefacts_images = []
for i in range(len(zooniverse_df['annotations'])):
    if zooniverse_df['annotations'][i][0]['task_label'] == 'Does the image have artefacts in it?' and \
     zooniverse_df['annotations'][i][0]['value'] in ['The image has artefacts. ', 'The image has **NO** artefacts.']:
        subject_data_key = list(zooniverse_df['subject_data'][i])[0] # Python 3.7 or later
        subject_data_key = zooniverse_df['subject_data'][i][subject_data_key]
        if 'metadata' in subject_data_key:
            image_name = subject_data_key['metadata'] 
        elif 'Observation' in subject_data_key:
            image_name = subject_data_key['Observation'] 
            annot = zooniverse_df['annotations'][i]
            if len(annot) <=1: # usually no annotations in the image or 'The image has **NO** artefacts.'
                if image_name not in no_artefacts_images:
                    no_artefacts_images.append(image_name)
            else:
                image = cv2.imread(images_dir+image_name+'.png')
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                masks, gt_classes, boxes = [], [], []
                for i in range(len(annot[1]['value'])):
                    value_key = annot[1]['value'][i]
                    tool_label = value_key['tool_label']
                    tool = value_key['tool']
                    frame = value_key['frame']
                    details = value_key['details']

                    # 0:Polygon
                    # 1:Bounding box (useful for **read-out-streaks**)
                    # 2:Free form
                    print(details[0]['value'])

                    if details[0]['value'] is not None:
                        print('label', values_dict[details[0]['value']])
                        gt_classes.append(values_dict[details[0]['value']])
                    else:
                        print('Label is None!!')
                        gt_classes.append('noone')
                    if tool in [0, 2]:
                        points = value_key['points']
                        pts = np.array([[point['x']/1024.0*512, point['y']/1024.0*512] for point in points])
                        pts = np.append(pts, [pts[0]], axis=0)
                        pts = np.around(pts) # mathematically round coords
                        pts = np.array(pts, dtype=np.int32)
                        
                        shape = (512, 512)  
                        mask = np.zeros(shape, np.int16)
                        cv2.fillPoly(mask, [pts], color=(255))
                        masks.append(mask)
                        mask_to_bbox = dataset_utils.mask_to_bbox(mask) # results in XYXY
                        boxes.append(mask_to_bbox)

                        # plt.imshow(mask)  # Display the mask in grayscale
                        # plt.title('Mask from Points')
                        # plt.axis('off')  # Hide axis ticks and labels
                        # plt.show()
                    else: 
                        bbox_coords = [value_key['x']/1024.0*512, value_key['y']/1024.0*512, value_key['width']/1024.0*512, value_key['height']/1024.0*512]
                        bbox_coords = np.around(bbox_coords) # mathematically round coords
                        bbox_coords = np.array(bbox_coords, dtype=np.int16)
                        mask_shape = (512, 512) 
                        mask = np.zeros(mask_shape, dtype=np.uint8)
                        
                        # Draw a filled rectangle on the mask where the bounding box is
                        cv2.rectangle(
                            mask, 
                            (bbox_coords[0], bbox_coords[1]), 
                            (bbox_coords[2]+bbox_coords[0], bbox_coords[3]+bbox_coords[1]), 
                            color=(255), 
                            thickness=cv2.FILLED)

                        masks.append(mask)
                        boxes.append([bbox_coords[0], bbox_coords[1], bbox_coords[2]+bbox_coords[0], bbox_coords[3]+bbox_coords[1]])
                        
                        # # Display the mask
                        # plt.imshow(mask)
                        # plt.title('Mask from Bounding Box')
                        # plt.axis('off')
                        # plt.show()

                    # gather info per image
                    images_dict[image_name+'.png'] = {'masks': np.array(masks), 
                                                      'boxes': np.array(boxes), 
                                                      'classes':np.array(gt_classes)}
                    
                    print(tool, annot[1]['value'][i].keys(), tool_label)
        else:
            print("NO metadata", zooniverse_df['annotations'][i])
            
        if subject_data_key['retired'] == None:
            print('Retired', image_name)
            continue
        

In [ ]:
images_dict#['S0742830141_L.png']['masks'].shape

In [ ]:
import glob
from roboflow import Roboflow
import os
from dataset import voc_annotate_and_Roboflow_export
import cv2
export_to_Roboflow = True

if export_to_Roboflow:
    # Initialize Roboflow client
    rf = Roboflow(api_key="EBeK30tpU3HW2VGGl0xa")
    upload_project = rf.workspace("iuliaelisa").project("xmm_om_artefacts_512") # error if the project doesn't exist
    
def export_image_det_to_Roboflow(input_dir, filename, masks, classes, boxes):
    
    objects = []
    for i in range(len(masks)):
        mask_np = masks[i]/255 # [H, W]
        polygon = voc_annotate_and_Roboflow_export.binary_image_to_polygon(mask_np)
        objects.append({
            'name': classes[i],
            'bbox': boxes[i],
            'segmentations': polygon[0]
        })
        
    if len(objects)>0:
        voc_annotate_and_Roboflow_export.create_annotation_SAM(
            filename=filename, 
            width=512, 
            height=512, 
            depth=3, 
            objects=objects, 
            offset= 1.0) # generating xml file for VOC format
        image_path = input_dir+filename
        annotation_filename = filename.replace(".png", ".xml")
        upload_project.upload(image_path, annotation_filename, overwrite=False)
        os.remove(annotation_filename)
    else:
        print("No objects after label filtering.")
    
for image_filename, annots in images_dict.items():
    print(image_filename)
    image_path = os.path.join(images_dir, image_filename)
    new_im = cv2.imread(image_path)
    if export_to_Roboflow:
        export_image_det_to_Roboflow(images_dir, image_filename, annots['masks'], annots['classes'], annots['boxes'])
    
    plt.imshow(new_im)
    plt.title('Detectron2 Masks')
    dataset_utils.show_masks(annots['masks']/255, plt.gca()) #, random_color=True)
    # plt.savefig('./plots/detr2_masks.png')
    plt.show()
    plt.close()

print("Segmentation of all images completed.")

In [23]:
# also, upload image with no artefacts

for img in no_artefacts_images:
# Upload the image to your project
    upload_project.upload(images_dir+img+".png")

In [61]:
no_artefacts_images

['S0674210201_L',
 'S0123920101_S',
 'S0785102401_S',
 'S0863230101_M',
 'S0677620137_M',
 'S0677660101_M',
 'S0504101601_S',
 'S0311590801_S',
 'S0414190701_M',
 'S0084230701_S',
 'S0651910701_M',
 'S0891801501_M',
 'S0112681301_S',
 'S0677590140_M',
 'S0552340101_M',
 'S0148740201_S',
 'S0556212201_M',
 'S0551690401_L',
 'S0741390301_S',
 'S0111290201_M',
 'S0827041801_M',
 'S0841951701_M',
 'S0151750101_M',
 'S0201130201_M',
 'S0500670301_M',
 'S0312190201_M',
 'S0156960201_S',
 'S0057740301_S',
 'S0412580301_S',
 'S0800761701_M',
 'S0205070301_M',
 'S0552170301_S',
 'S0721630101_S',
 'S0604960101_S',
 'S0111280301_M',
 'S0729160601_M',
 'S0510390201_S',
 'S0556215201_M',
 'S0785101701_M',
 'S0782500201_M',
 'S0823432101_M',
 'S0827040101_S',
 'S0677590131_M',
 'S0650560401_L',
 'S0785110101_M',
 'S0501370101_U',
 'S0842110101_S',
 'S0105860101_S']